# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import re

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'orders'

connection_string = f"{driver}//{user}:{password}@{ip}/{database}"

engine = create_engine(connection_string)


In [31]:
query = """
SELECT *
FROM orders
"""

orders = pd.read_sql(query, connection_string)


In [32]:
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [33]:
orders.shape

(397924, 14)

In [34]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
index           397924 non-null int64
InvoiceNo       397924 non-null int64
StockCode       397924 non-null object
year            397924 non-null int64
month           397924 non-null int64
day             397924 non-null int64
hour            397924 non-null int64
Description     397924 non-null object
Quantity        397924 non-null int64
InvoiceDate     397924 non-null object
UnitPrice       397924 non-null float64
CustomerID      397924 non-null int64
Country         397924 non-null object
amount_spent    397924 non-null float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


In [ ]:
'''Notes:
1- Column's names can be improved by using only lowercase letters.
2- No null values which is good.
3- Change Invoice_date to datetime64[ns]
4- Drop the index column
'''

In [35]:
orders.columns

Index(['index', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

In [28]:
columns_names = ['index','invoice_No', 'stock_code', 'year', 'month', 'day', 'hour',
       'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id',
       'country', 'amount_spent']

In [83]:
orders_clean = orders

In [84]:
orders_clean.columns = columns_names
orders_clean = orders.astype({'invoice_date':'datetime64[ns]'})

In [85]:
orders_clean = orders_clean.drop('index', axis = 1)

In [86]:
orders_clean.head()

,invoice_No,stock_code,year,month,day,hour,description,quantity,invoice_date,unit_price,customer_id,country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here

In [87]:
#Sub Problem 1: How to aggregate the  amount_spent for unique customers?

total_cust_amount = orders_clean.groupby('customer_id').sum()
total_cust_amount = total_cust_amount.drop(total_cust_amount.iloc[:,:7], axis =1)
total_cust_amount.columns = ['t_spent_by_cust']

In [88]:
total_cust_amount.head()

,t_spent_by_cust
customer_id,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [68]:
total_cust_amount_quantile = total_cust_amount.quantile([0.75,0.95])

In [69]:
total_cust_amount_quantile

,t_spent_by_cust
0.75,1661.640
0.95,5840.182


In [131]:
vip_customers = total_cust_amount[total_cust_amount['t_spent_by_cust']>= 5840.182]
vip_customers.columns = ['t_spent_by_vips']

In [132]:
vip_customers.head()

,t_spent_by_vips
customer_id,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53


In [137]:
preferred_cust = total_cust_amount[np.logical_and(total_cust_amount['t_spent_by_cust']>= 1661.640
                                                  , total_cust_amount['t_spent_by_cust'] < 5840.182)]
                                        
preferred_cust.columns = ['t_spent_by_pcs']

In [138]:
preferred_cust.head()

,t_spent_by_pcs
customer_id,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here

In [159]:
pc_ids = list(preferred_cust.index)
vip_ids = list(vip_customers.index)

country_vips = orders_clean[orders_clean.customer_id.isin(vip_ids)]

In [161]:
country_vips = country_vips.groupby('country').count()
country_vips = country_vips.drop(country_vips.iloc[:,1:], axis = 1)
country_vips.columns = ['total_vips_per_country']
country_vips = country_vips.sort_values('total_vips_per_country', ascending = False)

In [162]:
country_vips_country.head()

,total_vips_per_country
country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [152]:
# your code here

customer_id_vips_pcs = pc_ids + vip_ids

In [154]:
country_vips_pcs = orders_clean[orders_clean.customer_id.isin(customer_id_vips_pcs)]

In [163]:
country_vips_pcs = country_vips_pcs.groupby('country').count()
country_vips_pcs = country_vips_pcs.drop(country_vips_pcs.iloc[:,1:], axis = 1)
country_vips_pcs.columns = ['total_vips_pcs_per_country']
country_vips_pcs = country_vips_pcs.sort_values('total_vips_pcs_per_country', ascending = False)

In [164]:
country_vips_pcs.head()

,total_vips_pcs_per_country
country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
